### Import packges

In [3]:
%pip install py-algorand-sdk

  Using cached py_algorand_sdk-2.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached PyNaCl-1.5.0-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (856 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
Using cached py_algorand_sdk-2.5.0-py3-none-any.whl (111 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.0/559.0 kB 1.2 MB/s eta 0:00:0000:0100:01m
Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.6/477.6 kB 8.6 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))

from kmd_wallet import KmdAlgorand
kmd_algo = KmdAlgorand()

### create wallets

In [8]:
DEFAULT_KMD_WALLET_NAME = "unencrypted-default-wallet"
DEFAULT_KMD_WALLET_PASSWORD = ""
default_wallet = kmd_algo.create_user_wallet(wallet_name=DEFAULT_KMD_WALLET_NAME, wallet_password=DEFAULT_KMD_WALLET_PASSWORD)

print('here')
user_wallet = kmd_algo.create_user_wallet(wallet_name="naty", wallet_password="naty")
kmd_algo.get_account_address(user_wallet)

URLError: <urlopen error [Errno 111] Connection refused>

### Get wallets accounts list

In [9]:
default_wallet.list_keys()

NameError: name 'default_wallet' is not defined

In [4]:
user_wallet.list_keys()

['76IMCJF25JN6PRRCRECQ67VK2K4D3XZH7W35J44AM7LO2QBICK5QONKHHY']

### Check the balance of the accounts

In [8]:
default_account = default_wallet.list_keys()[0]
user_account = user_wallet.list_keys()[0]

In [35]:
# default account balance
kmd_algo.query_account_information(default_account)

{'address': 'E233ZFSBURTANGXL4VCBCKS7KSGCD2FCOFGBPCIXMML5OTYL7GRH6SFBHU',
 'amount': 3999997499975000,
 'amount-without-pending-rewards': 3999997499975000,
 'apps-local-state': [],
 'apps-total-schema': {'num-byte-slice': 0, 'num-uint': 0},
 'assets': [],
 'created-apps': [],
 'created-assets': [],
 'min-balance': 100000,
 'participation': {'selection-participation-key': 'cJaw20sosTc/VMJZVTllYs2n4YDMItvgHaNzWUAjfZU=',
  'state-proof-key': '8m9YoOyFe/SZNmRxEoYPkTv7rCmZpIcA1bI7e7GVDVHRxcBi0RtGiU7il040c8caJGgYTBucFMvK+QRIaAlhpA==',
  'vote-first-valid': 0,
  'vote-key-dilution': 10000,
  'vote-last-valid': 30000,
  'vote-participation-key': 'zBZTrRWzZHAiklhqrynWWRZuA4ndJYpBxbPI4nbyoDk='},
 'pending-rewards': 0,
 'reward-base': 0,
 'rewards': 0,
 'round': 33,
 'status': 'Online',
 'total-apps-opted-in': 0,
 'total-assets-opted-in': 0,
 'total-created-apps': 0,
 'total-created-assets': 0}

In [37]:
kmd_algo.query_account_information(user_account)

{'address': '76IMCJF25JN6PRRCRECQ67VK2K4D3XZH7W35J44AM7LO2QBICK5QONKHHY',
 'amount': 0,
 'amount-without-pending-rewards': 0,
 'apps-local-state': [],
 'apps-total-schema': {'num-byte-slice': 0, 'num-uint': 0},
 'assets': [],
 'created-apps': [],
 'created-assets': [],
 'min-balance': 100000,
 'pending-rewards': 0,
 'reward-base': 0,
 'rewards': 0,
 'round': 33,
 'status': 'Offline',
 'total-apps-opted-in': 0,
 'total-assets-opted-in': 0,
 'total-created-apps': 0,
 'total-created-assets': 0}

### Transfer algos from default account to user account

In [43]:
from algosdk.v2client import algod
# Specify the node address and token.

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# Initialize an algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address)

In [44]:
from algosdk import transaction

def send_alogs_transaction(algod_client, receiver_address, sender_address, sender_private_key, amount):

    # Construct the transaction
    params = algod_client.suggested_params()

    txn = transaction.PaymentTxn(
        sender=sender_address,
        receiver=receiver_address,
        amt=amount,
        sp=params
    )

    # Sign the transaction with the sender's private key
    signed_txn = txn.sign(sender_private_key)

    # Send the transaction to the Algorand network
    try:
        tx_id = algod_client.send_transaction(signed_txn)
        print("Transaction ID:", tx_id)
    except Exception as e:
        print("Transaction failed:", e)


In [45]:
sender_private_key = default_wallet.export_key(default_account)

In [46]:
send_alogs_transaction(
    algod_client=algod_client, 
    receiver_address=user_account, 
    sender_address=default_account, 
    sender_private_key=sender_private_key, 
    amount=1000000)

Transaction ID: AKEN67232DAJVDXQ6F2MAIAHNT34NOGU5I7LC656SZYVFVY2F2LA


### Check the balance of the user

In [47]:
kmd_algo.query_account_information(user_account)

{'address': '76IMCJF25JN6PRRCRECQ67VK2K4D3XZH7W35J44AM7LO2QBICK5QONKHHY',
 'amount': 1100000,
 'amount-without-pending-rewards': 1100000,
 'apps-local-state': [],
 'apps-total-schema': {'num-byte-slice': 0, 'num-uint': 0},
 'assets': [],
 'created-apps': [],
 'created-assets': [],
 'min-balance': 100000,
 'pending-rewards': 0,
 'reward-base': 0,
 'rewards': 0,
 'round': 35,
 'status': 'Offline',
 'total-apps-opted-in': 0,
 'total-assets-opted-in': 0,
 'total-created-apps': 0,
 'total-created-assets': 0}

### Create asset

In [48]:
def create_asset(sender_address, sender_private_key, asset_url):
    sp = algod_client.suggested_params()
    txn = transaction.AssetConfigTxn(
        sender=sender_address,
        sp=sp,
        default_frozen=False,
        unit_name="rug",
        asset_name="Really Useful Gift",
        manager=sender_address,
        reserve=sender_address,
        freeze=sender_address,
        clawback=sender_address,
        url=asset_url,
        total=1000,
        decimals=0,
    )

    # Sign with secret key of creator
    stxn = txn.sign(sender_private_key)
    # Send the transaction to the network and retrieve the txid.
    txid = algod_client.send_transaction(stxn)
    print(f"Sent asset create transaction with txid: {txid}")
    # Wait for the transaction to be confirmed
    results = transaction.wait_for_confirmation(algod_client, txid, 4)
    print(f"Result confirmed in round: {results['confirmed-round']}")
    return results

In [50]:
sender_private_key = user_wallet.export_key(user_account)

In [51]:
create_asset(
    sender_address=user_account, 
    sender_private_key=sender_private_key, 
    asset_url="https://path/to/my/asset/details")

Sent asset create transaction with txid: VC3BEFTNWGAFCNV6CEKELGOWEHCAERRJTUMJRIEPTD6EKWQLGVUQ
Result confirmed in round: 36


{'asset-index': 1036,
 'confirmed-round': 36,
 'pool-error': '',
 'txn': {'sig': 'YJmPVw19UkdwuMi5jz9KlUj3x46/Gpfa6IjbMjwPiXfMKFYbHGl8WK5Yc4eXMdbE/LSrYtEoSoTQF2c7fwnuCQ==',
  'txn': {'apar': {'an': 'Really Useful Gift',
    'au': 'https://path/to/my/asset/details',
    'c': '76IMCJF25JN6PRRCRECQ67VK2K4D3XZH7W35J44AM7LO2QBICK5QONKHHY',
    'f': '76IMCJF25JN6PRRCRECQ67VK2K4D3XZH7W35J44AM7LO2QBICK5QONKHHY',
    'm': '76IMCJF25JN6PRRCRECQ67VK2K4D3XZH7W35J44AM7LO2QBICK5QONKHHY',
    'r': '76IMCJF25JN6PRRCRECQ67VK2K4D3XZH7W35J44AM7LO2QBICK5QONKHHY',
    't': 1000,
    'un': 'rug'},
   'fee': 1000,
   'fv': 35,
   'gen': 'dockernet-v1',
   'gh': 'oLccfqHgI2WWmPc2Q/RqC5JO3vUfoBRcRLItBflYebo=',
   'lv': 1035,
   'snd': '76IMCJF25JN6PRRCRECQ67VK2K4D3XZH7W35J44AM7LO2QBICK5QONKHHY',
   'type': 'acfg'}}}